In [2]:
# Install dependencies
!pip install gradio scikit-learn pandas seaborn

# Import libraries
import gradio as gr
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder

# Load dataset
df = sns.load_dataset("mpg").dropna()  # we'll use this as a house-like dataset stand-in
df = df.rename(columns={"weight": "area", "horsepower": "bedrooms", "acceleration": "bathrooms"})

# Use 'mpg' as a pseudo price
df = df[["area", "bedrooms", "bathrooms", "mpg"]]

# Bin prices (mpg) into categories: Low, Medium, High
bins = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
df['price_category'] = bins.fit_transform(df[['mpg']]).astype(int)
label_map = {0: "Low", 1: "Medium", 2: "High"}
df['price_category'] = df['price_category'].map(label_map)

# Prepare training data
X = df.drop(columns=["mpg", "price_category"])
y = df["price_category"]
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Train model
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Prediction function
def predict_house_price(area, bedrooms, bathrooms):
    input_data = [[area, bedrooms, bathrooms]]
    prediction = model.predict(input_data)[0]
    return f"Predicted Price Category: {encoder.inverse_transform([prediction])[0]}"

# Create Gradio UI
interface = gr.Interface(
    fn=predict_house_price,
    inputs=[
        gr.Slider(minimum=1500, maximum=6000, label="Area (sq ft)", value=3000),
        gr.Slider(minimum=50, maximum=250, label="Bedrooms (approx horsepower)", value=100),
        gr.Slider(minimum=8, maximum=25, label="Bathrooms (approx acceleration)", value=15),
    ],
    outputs="text",
    title="🏠 House Price Classifier",
    description="Predicts house price category (Low, Medium, High) using Logistic Regression."
)

# Launch the app
interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ba70c5acd7ad4e4cde.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
